# Serial control of AD9910 through Arduino

In order to control AD9910 on its official [evaluation board](https://www.analog.com/en/design-center/evaluation-hardware-and-software/evaluation-boards-kits/eval-ad9910.html) through a serial (3-wire SPI) connection, set the board as follows:

Import packages:

In [1]:
import pyvisa

Connect to the Arduino:

In [2]:
rm = pyvisa.ResourceManager()
a = rm.open_resource('ASRL13::INSTR')

('ASRL8::INSTR', 'ASRL13::INSTR')

Write and test the function that prints the contents of all 23 registers:

In [96]:
def read_registers(arduino):
    print(arduino.query('r'), end='')
    for i in range(23):
        print(arduino.read(), end='')
        
read_registers(a)

Write functions to reset the AD9910, and to write register contents:

In [75]:
def reset(arduino):
    print(arduino.query('0'))
    
def write_register(arduino, reg, data_list, printing=False):
    # send the write command
    arduino.write_raw(
        b"w" \
          + bytes(bytearray([reg])) \
          + bytes(bytearray(data_list))
      )
    
    # print Arduino's return text
    if printing:
        print(arduino.read(), end='')
        print(arduino.read(), end='')

The following writes the register values that cause the AD9910 to output an 85 MHz sine wave for a 1 GHz clock input.

Note in particular the MSB of the second least significant byte in register 0x02, "REFCLK input divider bypass", which, when set to 1, disables the internal clock divider.

The amplitude, phase, and frequency settings of profile 0 are written in the 0x0E register.

In [125]:
write_register(a, 0x00, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x01, [0b00000000,0b01000000,0b00001000,0b00100000,])
write_register(a, 0x02, [0b00011111,0b00111111,0b11000000,0b00000000,])
write_register(a, 0x03, [0b00000000,0b00000000,0b01111111,0b01111111,])
write_register(a, 0x04, [0b11111111,0b11111111,0b11111111,0b11111111,])
write_register(a, 0x05, [0b11101001,0b00100000,0b01100100,0b10001001,0b11000100,0b11100000,])
write_register(a, 0x06, [0b10000111,0b00101100,0b00000101,0b10101001,0b10000000,0b00101001,])
write_register(a, 0x07, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x08, [0b00000000,0b00000000,])
write_register(a, 0x09, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0A, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0B, [0b10100001,0b10011001,0b11000100,0b00011010,0b00011001,0b00101110,0b00000011,0b11000100,])
write_register(a, 0x0C, [0b11000100,0b10101001,0b10110000,0b10000101,0b00000011,0b11000000,0b10000011,0b10110000,])
write_register(a, 0x0D, [0b01001110,0b11010000,0b00000000,0b00011100,])
write_register(a, 0x0E, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x0F, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x10, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x11, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x12, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x13, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x14, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x15, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x16, [0b00111000,0b11010000,0b01010011,0b11000111,])

Writing to register 0:   
  00000000,00000000,00000000,00000000,
Writing to register 1:   
  00000000,01000000,00001000,00100000,
Writing to register 2:   
  00011111,00111111,11000000,00000000,
Writing to register 3:   
  00000000,00000000,01111111,01111111,
Writing to register 4:   
  11111111,11111111,11111111,11111111,
Writing to register 5:   
  11101001,00100000,01100100,10001001,11000100,11100000,
Writing to register 6:   
  10000111,00101100,00000101,10101001,10000000,00101001,
Writing to register 7:   
  00000000,00000000,00000000,00000000,
Writing to register 8:   
  00000000,00000000,
Writing to register 9:   
  00000000,00000000,00000000,00000000,
Writing to register A:   
  00000000,00000000,00000000,00000000,
Writing to register B:   
  10100001,10011001,11000100,00011010,00011001,00101110,00000011,11000100,
Writing to register C:   
  11000100,10101001,10110000,10000101,00000011,11000000,10000011,10110000,
Writing to register D:   
  01001110,11010000,00000000,00011100,


Close connection to the Arduino:

In [93]:
a.close()